In [3]:
## Content for loading data

In [1]:
import pandas as pd
import numpy as np

def load_data(path):
    df = pd.read_excel(path)
    return df

In [5]:
## Content for preprocessing.py

In [2]:
def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration


def create_preprocess_date_time(df):
    '''
    This Function preprocess date_of_journey and duration to create departure and arrival date time.
    '''
    df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
    df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
    df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
    df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
    df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']
    return df

stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}

## Running the process of prediction is also referred as inference:
def sanity_check(df,train=True):
    '''
    This function performs sanity check on the airline data.
    inputs:
        df: dataframe that we need to perform sanity check
        train: This is used for process of training and inference.
            train is having default value of True and can be set as False if we are running inference.
            
        ## process for training    
        sanity_check(df)
        # process of prediction
        sanity_check(df,train=False)
    returns:
        df
    '''
    if train:
        df.drop_duplicates(inplace=True)
    
    create_preprocess_date_time(df)
    df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)
    df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)
    
    return df

In [3]:
def handle_missing_value(df,train=True):
    """
    This function helps to handle missing value.
    Since for Airline data there is just one missing value we can choose to drop missing value.
    inputs:
         df: dataframe which requires imputation.
         
    returns:
        df
    
    """
    df.dropna(inplace=True)
    return df

In [4]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict


def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict


from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict


## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict


def airline_handle_categorical_data(df,target):
    df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})
    df, encoded_dict = handle_categorical_values(df,target)
    categorical_cols = df.select_dtypes(object).columns
    df.drop(columns=categorical_cols,inplace=True)
    return df, encoded_dict


In [5]:
## Content for feature Engineering

In [6]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)


def time_based_feature_Engineering(df):

    df['dep_hr'] = df['DepartureDateTime'].dt.hour
    df['arr_hr'] = df['ArrivalDateTime'].dt.hour

    df['dep_month'] = df['DepartureDateTime'].dt.month
    df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

    df['arr_month'] = df['ArrivalDateTime'].dt.month
    df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

    df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
    df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

    df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
    df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


    df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
    df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
    df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
    df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

    one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
    df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
    new_df = pd.concat([df,df_oneHotEncoded],axis=1)
    
    drop_cols = [
                'DepartureDateTime',
                'Duration_timedelta',
                'ArrivalDateTime',
                'Freq_encoded_Source',
                'Mean_encoded_Source',
                'Mean_encoded_Destination',
                'departure_timeOfDay_encoded',
                'arr_month', 
                'arr_day_of_month',
                'departure_timeOfDay_early_morning',
                'departure_timeOfDay_evening',
                'arrival_timeOfDay_late_night',
                'arrival_timeOfDay_morning', 
                'arrival_timeOfDay_night']
    
    new_df.drop(columns=one_hot_cols,inplace=True)
    new_df.drop(columns=drop_cols,inplace=True)
    return new_df

In [7]:
## content for model model.py

In [8]:
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn import metrics

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import SplineTransformer

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


def regression(X_train, X_test, y_train, y_test):
    ## Baseline model
    reg = linear_model.LinearRegression()
    reg.fit(X_train,y_train)
    print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))
    return reg

def knearestneighbour(X_train, X_test, y_train, y_test):
    knn_regressor = KNeighborsRegressor(n_neighbors=10)
    knn_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,knn_regressor.predict(X_train)),metrics.r2_score(y_test,knn_regressor.predict(X_test)))
    return knn_regressor

def decisiontree(X_train, X_test, y_train, y_test):
    tree = DecisionTreeRegressor(max_depth=10)
    tree.fit(X_train,y_train)
    print(metrics.r2_score(y_train,tree.predict(X_train)),metrics.r2_score(y_test,tree.predict(X_test)))
    return tree

def randomForest(X_train, X_test, y_train, y_test):
    random_regressor = RandomForestRegressor(n_estimators=500, min_samples_split=10,min_samples_leaf=2)
    random_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,random_regressor.predict(X_train)),metrics.r2_score(y_test,random_regressor.predict(X_test)))
    return random_regressor

from xgboost import XGBRegressor
def xgboost(X_train, X_test, y_train, y_test):
    xgboost_regressor = XGBRegressor()
    xgboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))
    return xgboost_regressor

from catboost import  CatBoostRegressor
def catboost(X_train, X_test, y_train, y_test):
    catboost_regressor = CatBoostRegressor(verbose=False)
    catboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,catboost_regressor.predict(X_train)),metrics.r2_score(y_test,catboost_regressor.predict(X_test)))
    return catboost_regressor

from sklearn.ensemble import GradientBoostingRegressor
def gboost(X_train, X_test, y_train, y_test):
    gboost_regressor = GradientBoostingRegressor()
    gboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,gboost_regressor.predict(X_train)),metrics.r2_score(y_test,gboost_regressor.predict(X_test)))
    return gboost_regressor


def train(X,y,modelType):
    # Split your dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    model = modelType(X_train, X_test, y_train, y_test)
    return model

import pickle

def save_pickle(file_path,obj):
    with open(file_path,'wb') as file:
            pickle.dump(obj,file)
        
def load_pickle(file_path):
    with open(file_path,'rb') as file:
            obj = pickle.load(file)
    return obj

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
## train.py

In [63]:
target ='Price'

train_data_path = "../data/Train.xlsx"
model_path = '../models/xgboost_demo.pickle'
encoded_path = '../models/encoded_dict_demo.pickle'

## process for training   
df = load_data(train_data_path)
sanity_check(df)
handle_missing_value(df)

df, encoded_dict = airline_handle_categorical_data(df,target)
final_df = time_based_feature_Engineering(df)
X = final_df.drop(columns=target)
y = final_df[target]

In [64]:
X.columns

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Source', 'Label_encoded_Destination',
       'Label_encoded_Route', 'dep_hr', 'arr_hr', 'dep_month',
       'dep_day_of_month', 'dep_day_of_week', 'arr_day_of_week', 'dep_weekday',
       'arr_weekday', 'arrival_timeOfDay_encoded',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening'],
      dtype='object')

In [20]:
model = train(X,y,xgboost)
save_pickle(encoded_path,encoded_dict)
save_pickle(model_path,model)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.9476763358802425 0.8387199519385058


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [33]:
len(X.columns)

27

In [ ]:
## to predict.py

In [48]:
test_data_path = "../data/Test.xlsx"
model_path = '../models/xgboost_demo.pickle'
encoded_path = '../models/encoded_dict_demo.pickle'

# process of prediction
df = load_data(test_data_path)
df= df.iloc[:5,:]

In [49]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

In [50]:
sanity_check(df,train=False)
df.columns

Index(['Airline', 'Source', 'Destination', 'Route', 'Total_Stops',
       'DepartureDateTime', 'Duration_min', 'Duration_timedelta',
       'ArrivalDateTime'],
      dtype='object')

In [ ]:
loaded_dict = load_pickle(encoded_path)

# transform categorical values 
## write down a function.
def encode_predict_input(df,encoded_dict):
    '''
    This function encodes categorical values with same values as training encoded values.
    Input:
      df : DataFrame
      encoded_dict : Category encoded dictionary
    returns :None
    '''
    encoded_cols = ['Airline', 'Source', 'Destination', 'Route']
    
    frequency_dict = encoded_dict['Frequency']
    mean_dict = encoded_dict['Mean']
    label_dict = encoded_dict['Label']
    for col in encoded_cols:
        df["Freq_encoded_"+col] = df[col].replace(frequency_dict[col])
        df["Mean_encoded_"+col] = df[col].replace(mean_dict[col])
        df["Label_encoded_"+col] = df[col].replace(label_dict[col])
    df.drop(columns=encoded_cols,inplace=True)

    return df
        

In [55]:
df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})
df = encode_predict_input(df,loaded_dict)

In [56]:
df.columns

Index(['Total_Stops', 'DepartureDateTime', 'Duration_min',
       'Duration_timedelta', 'ArrivalDateTime', 'dep_hr', 'arr_hr',
       'dep_month', 'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'departure_timeOfDay', 'departure_timeOfDay_encoded',
       'arrival_timeOfDay', 'arrival_timeOfDay_encoded',
       'Freq_encoded_Airline', 'Mean_encoded_Airline', 'Label_encoded_Airline',
       'Freq_encoded_Source', 'Mean_encoded_Source', 'Label_encoded_Source',
       'Freq_encoded_Destination', 'Mean_encoded_Destination',
       'Label_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Route',
       'Label_encoded_Route'],
      dtype='object')

In [57]:
test_X = time_based_feature_Engineering(df)

In [59]:
len(df.columns)

31

In [60]:
len(test_X.columns)

24

In [61]:
X.columns

Index(['Total_Stops', 'Duration_min', 'dep_hr', 'arr_hr', 'dep_month',
       'dep_day_of_month', 'dep_day_of_week', 'arr_day_of_week', 'dep_weekday',
       'arr_weekday', 'arrival_timeOfDay_encoded', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Label_encoded_Airline', 'Label_encoded_Source',
       'Freq_encoded_Destination', 'Label_encoded_Destination',
       'Freq_encoded_Route', 'Mean_encoded_Route', 'Label_encoded_Route',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_morning',
       'arrival_timeOfDay_early_morning', 'arrival_timeOfDay_evening'],
      dtype='object')

In [66]:
list_A = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Freq_encoded_Destination', 'Freq_encoded_Route',
       'Mean_encoded_Airline', 'Mean_encoded_Route', 'Label_encoded_Airline',
       'Label_encoded_Source', 'Label_encoded_Destination',
       'Label_encoded_Route', 'dep_hr', 'arr_hr', 'dep_month',
       'dep_day_of_month', 'dep_day_of_week', 'arr_day_of_week', 'dep_weekday',
       'arr_weekday', 'arrival_timeOfDay_encoded',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening'] 
list_B = ['Total_Stops', 'Duration_min', 'dep_hr', 'arr_hr', 'dep_month',
       'dep_day_of_month', 'dep_day_of_week', 'arr_day_of_week', 'dep_weekday',
       'arr_weekday', 'arrival_timeOfDay_encoded', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Label_encoded_Airline', 'Label_encoded_Source',
       'Freq_encoded_Destination', 'Label_encoded_Destination',
       'Freq_encoded_Route', 'Mean_encoded_Route', 'Label_encoded_Route',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_morning',
       'arrival_timeOfDay_early_morning', 'arrival_timeOfDay_evening']

set(list_A) - set(list_B)

{'arrival_timeOfDay_after_noon',
 'departure_timeOfDay_late_night',
 'departure_timeOfDay_night'}

In [47]:
loaded_model = load_pickle(model_path)

loaded_model.predict(X)

ValueError: Feature shape mismatch, expected: 27, got 24

In [ ]:
## Error Encountered because of Routes not present in training set.

In [40]:
loaded_dict

{'Frequency': {'Airline': {'Air Asia': 0.030491301854329956,
   'Air India': 0.1619193270885108,
   'GoAir': 0.018543299560313517,
   'IndiGo': 0.1952781494934047,
   'Jet Airways': 0.35366086790288664,
   'Jet Airways Business': 0.0005735041101127892,
   'Multiple carriers': 0.1143184859491493,
   'Multiple carriers Premium economy': 0.0012425922385777097,
   'SpiceJet': 0.0779009749569872,
   'Trujet': 9.558401835213153e-05,
   'Vistara': 0.04568916077231887,
   'Vistara Premium economy': 0.0002867520550563946},
  'Source': {'Banglore': 0.20827757598929458,
   'Chennai': 0.03641751099216211,
   'Delhi': 0.41531255974001147,
   'Kolkata': 0.27337029248709616,
   'Mumbai': 0.06662206079143568},
  'Destination': {'Banglore': 0.27337029248709616,
   'Cochin': 0.41531255974001147,
   'Delhi': 0.20827757598929458,
   'Hyderabad': 0.06662206079143568,
   'Kolkata': 0.03641751099216211},
  'Route': {'BLR → AMD → DEL': 0.0017205123303383673,
   'BLR → BBI → DEL': 0.0004779200917606576,
   'BL

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10462 entries, 0 to 10682
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   Total_Stops                  10462 non-null  float64        
 1   Price                        10462 non-null  int64          
 2   DepartureDateTime            10462 non-null  datetime64[ns] 
 3   Duration_min                 10462 non-null  int64          
 4   Duration_timedelta           10462 non-null  timedelta64[ns]
 5   ArrivalDateTime              10462 non-null  datetime64[ns] 
 6   Freq_encoded_Airline         10462 non-null  float64        
 7   Freq_encoded_Source          10462 non-null  float64        
 8   Freq_encoded_Destination     10462 non-null  float64        
 9   Freq_encoded_Route           10462 non-null  float64        
 10  Mean_encoded_Airline         10462 non-null  float64        
 11  Mean_encoded_Source         

In [43]:
# process of prediction
df = load_data(test_data_path)
sanity_check(df,train=False)
loaded_dict = load_pickle(encoded_path)

In [44]:
encoded_cols = ['Route']
frequency_dict = encoded_dict['Frequency']
mean_dict = encoded_dict['Mean']
label_dict = encoded_dict['Label']
for col in encoded_cols:
    df["Freq_encoded_"+col] = df[col].replace(frequency_dict[col])
    df["Mean_encoded_"+col] = df[col].replace(mean_dict[col])
    df["Label_encoded_"+col] = df[col].replace(label_dict[col])

df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})
df = encode_predict_input(df,loaded_dict)

In [45]:
mean_dict['Route']['BLR → TRV → DEL']

KeyError: 'BLR → TRV → DEL'

In [46]:
df[df['Freq_encoded_Route'].str.contains('→',na=False)]

,Total_Stops,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Route,Freq_encoded_Airline,Mean_encoded_Airline,Label_encoded_Airline,Freq_encoded_Source,Mean_encoded_Source,Label_encoded_Source,Freq_encoded_Destination,Mean_encoded_Destination,Label_encoded_Destination
6,1,2019-12-03 07:30:00,905,0 days 15:05:00,2019-12-03 22:35:00,BLR → TRV → DEL,BLR → TRV → DEL,BLR → TRV → DEL,0.161919,9556.608028,1,0.208278,8022.872877,0,0.208278,8022.872877,2
72,3,2019-01-04 05:50:00,960,0 days 16:00:00,2019-01-04 21:50:00,CCU → IXR → BBI → BOM → BLR,CCU → IXR → BBI → BOM → BLR,CCU → IXR → BBI → BOM → BLR,0.161919,9556.608028,1,0.273370,9143.083566,3,0.273370,9143.083566,0
484,3,2019-03-24 05:50:00,960,0 days 16:00:00,2019-03-24 21:50:00,CCU → IXR → BBI → BOM → BLR,CCU → IXR → BBI → BOM → BLR,CCU → IXR → BBI → BOM → BLR,0.161919,9556.608028,1,0.273370,9143.083566,3,0.273370,9143.083566,0
966,2,2019-06-03 08:00:00,610,0 days 10:10:00,2019-06-03 18:10:00,BOM → VGA → TIR → HYD,BOM → VGA → TIR → HYD,BOM → VGA → TIR → HYD,0.161919,9556.608028,1,0.066622,5059.708752,4,0.066622,5059.708752,3
1838,1,2019-03-03 07:30:00,905,0 days 15:05:00,2019-03-03 22:35:00,BLR → TRV → DEL,BLR → TRV → DEL,BLR → TRV → DEL,0.161919,9556.608028,1,0.208278,8022.872877,0,0.208278,8022.872877,2
1980,2,2019-12-03 07:15:00,560,0 days 09:20:00,2019-12-03 16:35:00,BOM → IXC → DEL → HYD,BOM → IXC → DEL → HYD,BOM → IXC → DEL → HYD,0.353661,11599.021081,4,0.066622,5059.708752,4,0.066622,5059.708752,3
